In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import openai

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_openai import ChatOpenAI # paid model
from langchain_groq import ChatGroq #open source using Groq which provide model and LPU resources in cloud

In [7]:
model = ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)

In [8]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022DA8902440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022DA8903820>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
model2 = ChatOpenAI(model="gpt-4o")

In [10]:
model2

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000022DA8932080>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000022DA89541C0>, root_client=<openai.OpenAI object at 0x0000022DA8930220>, root_async_client=<openai.AsyncOpenAI object at 0x0000022DA89320E0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [12]:
from langchain_core.messages import HumanMessage,SystemMessage

In [14]:
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

In [15]:
model.invoke(messages)

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual greeting for someone you know well.)\n* **Coucou, ça va ?** (This is a very informal greeting, often used with close friends.)\n\nYou can choose the option that best suits the context and your relationship with the person you\'re greeting. \n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 21, 'total_tokens': 159, 'completion_time': 0.250909091, 'prompt_time': 0.000936588, 'queue_time': 0.035127488, 'total_time': 0.251845679}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c25e389f-0829-4047-b5c3-360b6131f3c9-0', usage_metadata={'

In [16]:
model2.invoke(messages)

AIMessage(content='Bonjour, comment ça va ?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 23, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_523b9b6e5f', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd244aec-fc50-41bb-9f1f-ed09d01f3a08-0', usage_metadata={'input_tokens': 23, 'output_tokens': 7, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
res = model.invoke(messages)

In [18]:
res

AIMessage(content='Hello: Bonjour\n\nHow are you?: Comment allez-vous? \n\n\nSo, together it would be:  Bonjour, comment allez-vous? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 21, 'total_tokens': 56, 'completion_time': 0.063636364, 'prompt_time': 0.00016916, 'queue_time': 0.018590826999999997, 'total_time': 0.063805524}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6b1dec74-160a-4ce3-88f5-a3e479b7fa9d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 35, 'total_tokens': 56})

In [19]:
from langchain_core.output_parsers import StrOutputParser

In [21]:
parser = StrOutputParser() # used to extract content from result
parser.invoke(res)

'Hello: Bonjour\n\nHow are you?: Comment allez-vous? \n\n\nSo, together it would be:  Bonjour, comment allez-vous? \n\n'

In [22]:
### using LCEL - chain components

chain = model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to someone you know well)\n* **Coucou, ça va ?** (Very informal, to someone you\'re close to)\n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n'

In [23]:
## instead of SystemMessage,HumanMessage everytime , we can use prompt templates, this is efficient way

In [24]:
from langchain_core.prompts import ChatPromptTemplate

In [27]:
prompt = ChatPromptTemplate.from_messages(
[("system","Translate the following in to {language}:"),("user","{text}")]
)

In [29]:
res = prompt.invoke({"language":"French","text":"Hello"})

In [30]:
res.to_messages()

[SystemMessage(content='Translate the following in to French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [31]:
chain = prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'